In [ ]:
!pip install -q ultralytics

In [ ]:
import os
import glob
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torchvision.ops import box_iou

In [ ]:
keyframes_dir = '/kaggle/input/keyframes-extra-aic2024-v3'
all_keyframe_paths = dict()
for part in sorted(os.listdir(keyframes_dir)):
    data_part = part #.split('_')[-1] # L01, L02 for ex
    print(data_part)
    all_keyframe_paths[data_part] =  dict()

for data_part in sorted(all_keyframe_paths.keys()):
    data_part_path = f'{keyframes_dir}/{data_part}/keyframes'
    video_dirs = sorted(os.listdir(data_part_path))
    video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

In [ ]:
all_keyframe_paths['Keyframes_L30'].keys()

In [ ]:
class VisualEncoding:
    def __init__(self,
                classes = ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
                           'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
                           'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
                           'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
                           'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
                           'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
                           'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
                           'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
                           'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
                           'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
                           'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
                           'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
                           'scissors', 'teddy bear', 'hair drier', 'toothbrush'),
                colors = ('black', 'blue', 'brown', 'green', 'grey', 'orange_', 'pink', 'purple',
                          'red', 'white', 'yellow'),
                row_str = ["0", "1", "2", "3", "4", "5", "6"],
                col_str = ["a", "b", "c", "d", "e", "f", "g"]):
        
        self.classes = classes
        self.colors = colors
        self.classes2idx = dict()
        for i, class_ in enumerate(classes):
            self.classes2idx[class_] = i
        self.n_row = len(row_str)
        self.n_col = len(col_str)

        x_pts = np.linspace(0, 1, self.n_row+1)
        y_pts = np.linspace(0, 1, self.n_col+1)

        self.grid_bboxes = []
        self.grid_labels = []
        for i in range(self.n_row):
            for j in range(self.n_col):
                label = col_str[j] + row_str[i]
                self.grid_bboxes.append([x_pts[j], y_pts[i], x_pts[j+1], y_pts[i+1]])
                self.grid_labels.append(label)

        self.grid_bboxes = np.array(self.grid_bboxes)

    def visualize_grid(self, grid_vis=None):
        if grid_vis is None:
            grid_vis = np.zeros((500, 500, 1))

        vis_h, vis_w, _ = grid_vis.shape
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 0.5
        color = (255, 0, 0)
        thickness = 2
        for i in range(self.n_row*self.n_col):
            x_start, y_start, x_end, y_end = self.grid_bboxes[i]
            label = self.grid_labels[i]
            org = (int((x_start + (x_end-x_start)/2)*vis_w), int((y_start + (y_end-y_start)/2)*vis_h))

            # Draw text
            grid_vis = cv2.putText(grid_vis, label, org, font, fontScale, color, thickness, cv2.LINE_AA)
            # Draw grid
            grid_vis = cv2.rectangle(grid_vis, (int(x_start*vis_w), int(y_start*vis_h)), (int(x_end*vis_w), int(y_end*vis_h)), color, thickness)
        plt.imshow(grid_vis)

    def encode_bboxes(self, bboxes, labels):
        '''
        Args:
            bboxes: np.array: (n_bboxes, 4) - expected normalized bbox in form (x0, y0, x1, y1)
            labels: np.array: (n_bboxes, )
        '''
        iou = box_iou(torch.as_tensor(bboxes), torch.as_tensor(self.grid_bboxes))
        bboxes_idx, locs_idx = np.nonzero(iou.numpy())

        context = []
        for bbox_idx, loc_idx in zip(bboxes_idx, locs_idx):
            context.append(self.grid_labels[loc_idx] + self.classes[labels[bbox_idx]].replace(" ", ""))
        context = ' '.join(map(str, context))
        return context

    def encode_classes(self, labels):
        '''
        Args:
            labels: np.array: (n_bboxes, )
        '''
        return ''
#         unique_classes, counts = np.unique(labels, return_counts=True)
#         context = []
#         for unique_class, count in zip(unique_classes, counts):
#             for i in range(count):
#                 context.append(self.classes[unique_class].replace(" ", "") + str(i))
#         context = ' '.join(map(str, context))
#         return context

    def encode_numbers(self, labels):
        '''
        Args:
            labels: np.array: (n_bboxes, )
        '''
        unique_classes, counts = np.unique(labels, return_counts=True)
        context = []
        for unique_class, count in zip(unique_classes, counts):
            context.append(self.classes[unique_class].replace(" ", "") + str(count))
        context = ' '.join(map(str, context))
        return context
  
    def encode_colors(self, bboxes, colors):
        '''
        Args:
            bboxes: np.array: (n_bboxes, 4) - expected normalized bbox in form (x0, y0, x1, y1)
            colors: np.array: (n_bboxes, )
        '''
        iou = box_iou(torch.as_tensor(bboxes), torch.as_tensor(self.grid_bboxes))
        bboxes_idx, locs_idx = np.nonzero(iou.numpy())

        context = []
        for bbox_idx, loc_idx in zip(bboxes_idx, locs_idx):
            context.append(self.grid_labels[loc_idx] + colors[bbox_idx].replace("_", ""))
        context = ' '.join(map(str, context))
        return context

    def encode(self, bboxes=None, labels=None, bboxes_colors=None, colors=None):
        '''
        Args:
            bboxes: np.array: (n_bboxes, 4) - expected normalized bbox in form (x0, y0, x1, y1)
            labels: np.array: (n_bboxes, )
        '''
        results = dict()
        if bboxes is not None:
            results['bbox'] = self.encode_bboxes(bboxes, labels)
            results['class'] = self.encode_classes(labels)
        else:
            results['bbox'] = results['class'] = None

        if bboxes_colors is not None:
            results['color'] = self.encode_colors(bboxes_colors, colors)
        else:
            results['color'] = None

        return results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO('yolov10x.pt')  # pretrained YOLOv8n model

In [ ]:
import os
from tqdm import tqdm
from PIL import UnidentifiedImageError

bs = 64
save_dirs = ["./context_encoded/bboxes_encoded",  "./context_encoded/number_encoded"] #"./context_encoded/classes_encoded",
encoder = VisualEncoding()

# Ensure all necessary directories exist
for save_dir in save_dirs:
    os.makedirs(save_dir, exist_ok=True)

for key, video_keyframe_paths in tqdm(all_keyframe_paths.items()):
    video_ids = sorted(video_keyframe_paths.keys())
    
    # Create subdirectories for each key inside the save_dirs
    for save_dir in save_dirs:
        key_dir = os.path.join(save_dir, key)
        os.makedirs(key_dir, exist_ok=True)
    
    for video_id in tqdm(video_ids):
        video_keyframe_path = video_keyframe_paths[video_id]
        video_encoded_bboxes, video_encoded_classes, video_encoded_numbers = [], [], []
        for i in range(0, len(video_keyframe_path), bs):
            image_paths = video_keyframe_path[i:i+bs]
            try:
                results = model(image_paths, conf=0.5, device=device, verbose=False)  # return a list of Results objects
            except Exception as e:
                print(e)
            
            for result in results:
                bboxes = result.boxes.xyxyn.cpu().numpy().copy()
                labels = result.boxes.cls.cpu().numpy().copy().astype(int)
                if len(bboxes) == 0:
                    video_encoded_bboxes.append("")
                    video_encoded_numbers.append("")
                else:
                    video_encoded_bboxes.append(encoder.encode_bboxes(bboxes, labels))
                    video_encoded_numbers.append(encoder.encode_numbers(labels))
        
        # Save the encoded data to the appropriate directories
        bbox_file_path = os.path.join(save_dirs[0], key, f"{video_id}.txt")
        number_file_path = os.path.join(save_dirs[1], key, f"{video_id}.txt")
        
        try:
            with open(bbox_file_path, "w") as f:
                for item in video_encoded_bboxes:
                    f.write("%s\n" % item)
            with open(number_file_path, "w") as f:
                for item in video_encoded_numbers:
                    f.write("%s\n" % item)
        except IOError as e:
            print(f"Error writing to file: {e}")

In [ ]:
!rm /kaggle/working/yolov10x.pt